# 👩‍💻 **Perform Lightweight Fine-Tuning with LoRA**

**Time Estimate:** 60 minutes

## 📋 **Overview**

In this lab, you will explore Lightweight Fine-Tuning using LoRA (Low-Rank Adaptation), a technique to efficiently adapt pre-trained language models. This method is particularly useful for cases with limited computational resources as it modifies only specific parameters instead of retraining the entire model. You will fine-tune a model for a sentiment classification task, using the IMDB dataset, which is an essential skill in deploying AI solutions with constrained resources.

## 🎯 **Learning Outcomes**

By the end of this lab, you will be able to:

- Apply LoRA to fine-tune a pre-trained language model for a targeted task.
- Evaluate the trade-offs between performance gains and computational efficiency.
- Understand the practical considerations when adapting large models with limited resources.

In [ ]:
# imports
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
import numpy as np

## Task 1: Dataset Preparation [15 minutes]

You will start by setting up the IMDB dataset for sentiment analysis, a common task in NLP.

1. Inspect the dataset structure. Think about:
   - What are the inputs and outputs?
   - How is the data labeled?
2. Analyze data distribution and consider if further manipulation is needed.

In [ ]:
# Task 1
# your code here...

✅ **Success Checklist**

- Dataset is loaded successfully.
- Training and test datasets are correctly identified.

💡 **Key Points**

- Understand dataset formats and load them correctly for usage.
- Consider label distribution and dataset integrity.

❗ **Common Mistakes to Avoid**

- Not exploring the dataset structure before proceeding with modeling.
- Failing to verify that the dataset splits are appropriate for your task.
- Ignoring data quality issues like missing values or inconsistent formatting.

## Task 2: Model Setup and Initialization [20 minutes]

Set up the environment and initialize a pre-trained transformer model to work with LoRA.

1. Ensure the required libraries are installed, such as Hugging Face Transformers and PyTorch.
2. Load the model and tokenizer without errors.

In [ ]:
# Task 2
# your code here ...

✅ **Success Checklist**

- Libraries are installed correctly.
- Model and tokenizer are successfully initialized.

💡 **Key Points**

- Familiarity with model and library initialization is crucial for starting any adaptation task.
- Make sure version compatibility between libraries is maintained.

❗ **Common Mistakes to Avoid**

- Using incompatible library versions that cause import errors.
- Not verifying model configuration matches your task requirements.
- Failing to handle potential CUDA/device compatibility issues.

## Task 3: Implement LoRA and Fine-Tune [40 minutes]

Apply the LoRA methodology to adapt and fine-tune your model on the training data.

1. Fine-tune the model using the training data.
2. During training, monitor for potential overfitting by analyzing evaluation metrics.

In [ ]:
# Task 3
# your code here...

✅ **Success Checklist**

- LoRA is configured and applied correctly.
- Model trains without errors and completes all epochs.

💡 **Key Points**

- LoRA allows for efficient adaptation by adjusting fewer model parameters.
- Fine-tuning requires careful setup of hyperparameters to balance learning.

❗ **Common Mistakes to Avoid**

- Setting LoRA rank too high, negating efficiency benefits.
- Not properly configuring the adapter for the specific model architecture.
- Ignoring memory constraints when setting batch sizes and sequence lengths.

🚀 **Next Steps**

In the next module, you will explore other fine-tuning strategies and evaluate their effectiveness for different machine learning challenges, enhancing your ability to make strategic model adaptations for specific scenarios.

## 💻 Exemplar Solution

<details>    
<summary><strong>Click HERE to see an exemplar solution</strong></summary>

### Task 1 Solution - Dataset Preparation
    
```python
from datasets import load_dataset
import pandas as pd

# Load IMDB dataset
dataset = load_dataset("imdb")

# Split dataset into training and test sets
train_dataset = dataset['train']
test_dataset = dataset['test']

# Explore dataset structure
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Features: {train_dataset.features}")

# Examine sample entries
print("\nSample entry:")
print(f"Text: {train_dataset[0]['text'][:200]}...")
print(f"Label: {train_dataset[0]['label']} (0=negative, 1=positive)")

# Check label distribution
train_labels = [example['label'] for example in train_dataset]
print(f"\nLabel distribution in training set:")
print(f"Negative (0): {train_labels.count(0)}")
print(f"Positive (1): {train_labels.count(1)}")

# Limit dataset size for faster training (optional)
train_dataset = train_dataset.select(range(5000))
test_dataset = test_dataset.select(range(1000))
print(f"\nUsing {len(train_dataset)} training and {len(test_dataset)} test samples")
```

### Task 2 Solution - Model Setup and Initialization

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,  # Binary classification for sentiment
    ignore_mismatched_sizes=True
)

# Move model to device
model.to(device)

# Test tokenizer with sample text
sample_text = "This movie was amazing!"
tokens = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)
print(f"\nSample tokenization:")
print(f"Input IDs shape: {tokens['input_ids'].shape}")
print(f"Attention mask shape: {tokens['attention_mask'].shape}")

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print(f"\nModel loaded successfully: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")
```

### Task 3 Solution - LoRA Implementation and Fine-Tuning

```python
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate
import numpy as np

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    r=8,  # Rank of adaptation
    lora_alpha=16,  # LoRA scaling parameter
    lora_dropout=0.1,  # LoRA dropout
    target_modules=["query", "value"]  # Target attention modules
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=512
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
print("Starting LoRA fine-tuning...")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"\nEvaluation Results: {eval_results}")

# Test prediction on sample text
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1)
    
    sentiment = "positive" if predicted_class.item() == 1 else "negative"
    confidence = predictions[0][predicted_class].item()
    
    return sentiment, confidence

# Test predictions
test_texts = [
    "This movie was absolutely fantastic!",
    "I hated this film, it was terrible.",
    "The movie was okay, nothing special."
]

print("\nSample Predictions:")
for text in test_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"Predicted: {sentiment} (confidence: {confidence:.3f})\n")
```
</details>